In [2]:
import pandas as pd
import numpy as np

from gensim.models import Word2Vec
from nltk.tokenize import sent_tokenize, word_tokenize
import re

import pickle

In [3]:
df_fake = pd.read_csv('data/Fake.csv')
df_true = pd.read_csv('data/True.csv')
df_fake['target'] = 0
df_true['target'] = 1
df = pd.concat([df_fake, df_true], ignore_index=True)
df_text = df.drop(['subject', 'date'], axis=1)
print(df_text.shape)
df_text.head()

(44898, 3)


,title,text,target
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,0


In [4]:
def tokenize(text_: str)->list:
    text_ = text_.replace("\n", " ") 
    tokenized_text = []
    # Итерируемся по каждому предложению в фалйе
    for i in sent_tokenize(text_):
        temp = []
        # Токенизируем предложения в слова
        sentence = i.lower()
        sentence = re.sub('[^a-zA-Z]', ' ', sentence)
        sentence=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",sentence)
        sentence=re.sub("(\\d|\\W)+"," ",sentence)
        
        for j in word_tokenize(sentence):
            temp.append(j.lower())
    
        tokenized_text.append(temp)
    return tokenized_text

In [19]:
COUNT_TEXTS = len(df_text)
all_texts = []
for i, row in enumerate(df_text.values):
    if i > COUNT_TEXTS:
        break
    
    title_ = row[0]
    text_ = row[1]
        
    title_list_ = tokenize(title_)
    text_list_ = tokenize(text_)
    
    all_texts.extend(title_list_)
    all_texts.extend(text_list_)
    
    print(f"building corpuses for model is: {i / COUNT_TEXTS*100:.2f}%", end='\r')
    

In [20]:
print(f"Количество предложений: {len(all_texts)}")

Количество предложений: 716816


In [21]:
#Save tokenized texts
with open('output/tokenize_texts.pkl', 'wb') as f:
    pickle.dump(all_texts, f)

In [22]:
#Load tokenized texts
with open('output/tokenize_texts.pkl', 'rb') as f:
    loaded_tokenized_text = pickle.load(f)

In [23]:
print(f"Количество предложений: {len(loaded_tokenized_text)}")

Количество предложений: 716816


In [24]:
model_w2v = Word2Vec(loaded_tokenized_text, min_count = 10, vector_size = 20, window = 10, seed=42)
print(len(model_w2v.wv.key_to_index))

32257


In [27]:
model_w2v.train(loaded_tokenized_text, total_examples=model_w2v.corpus_count, epochs=30, report_delay=0.1, compute_loss=True)

(444978149, 571038120)

In [28]:
#Сохраним модель W2V
model_w2v.save('output/models/model_w2v.model')

In [29]:
#Загрузим модель W2V
model_w2v = Word2Vec.load('output/models/model_w2v.model')

In [31]:
len(model_w2v.wv.key_to_index)

32257